# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [2]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [3]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [4]:
def P1(num_examples=5):
### STUDENT START ###
    for i in range(num_examples):
        print 'Label:' + categories[train_labels[i]]
        print 'Text:' + train_data[i] 
        print '------------------------------------------------'
        


### STUDENT END ###
P1(5)

Label:talk.religion.misc
Text:Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
------------------------------------------------
Label:sci.space
Text:

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrati

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [5]:
def P2():
## STUDENT START ###
    from sklearn.feature_extraction.text import CountVectorizer
    cvec = CountVectorizer()
#a
    t_train = cvec.fit_transform(train_data)
    vocab_size = len(cvec.vocabulary_)
    print "A: out of transform"
    print "Vocabulary size:", vocab_size
    print "avg num of nonzeros per example:", t_train.nnz * 1.0 / t_train.shape[0]
    print "fraction of non-zero entries in matrix: ", t_train.nnz*1.0 / (t_train.shape[0] * t_train.shape[1])
    print ''
#b
    featurenames = cvec.get_feature_names()
    print "B: 0th and last feature strings"
    print featurenames[0], featurenames[-1]
    print ''
#c

    myvocab =  ["atheism", "graphics", "space", "religion"]
    cvec_c = CountVectorizer(vocabulary=myvocab)
    t_train_c = cvec_c.fit_transform(train_data)
    vocab_size_c = len(cvec_c.vocabulary_)
    print "C: "
    print "Vocabulary Size:", vocab_size_c
    print "avg num of nonzeros per example:", t_train_c.nnz * 1.0 / t_train_c.shape[0]
    print ''

#d
    bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
    trigram_vectorizer = CountVectorizer(ngram_range=(1, 3))
    t_train_bigram = bigram_vectorizer.fit_transform(train_data)
    t_train_trigram = trigram_vectorizer.fit_transform(train_data)
    print "D: Bigram and Trigram"
    print 'Bigram Vocab: ', len(bigram_vectorizer.vocabulary_)
    print 'Trigram Vocab: ', len(trigram_vectorizer.vocabulary_)
    print ''
#e
    cvec_e = CountVectorizer(min_df=10)
    t_train_e = cvec_e.fit_transform(train_data)
    vocab_size_e = cvec_e.vocabulary_
    print "E: min_df=10"
    print 'min_df=10 vocab size:', len(vocab_size_e)
    print ''
#f
    print "F: diff between train and dev data vocab"
    cvec_f1 = CountVectorizer()
    cvec_f1.fit_transform(dev_data)
    cvec_f2 = CountVectorizer()
    cvec_f2.fit_transform(train_data)
    print "dev data vocab size:", len(cvec_f1.vocabulary_)
    print "train data vocab size:", len(cvec_f2.vocabulary_)
    print "train_data vocab size - dev_data vocab size: ", len(cvec_f2.vocabulary_) - len(cvec_f1.vocabulary_)
    print ''
    
## STUDENT END ###
P2()

A: out of transform
Vocabulary size: 26879
avg num of nonzeros per example: 96.7059980334
fraction of non-zero entries in matrix:  0.00359782722696

B: 0th and last feature strings
00 zyxel

C: 
Vocabulary Size: 4
avg num of nonzeros per example: 0.268436578171

D: Bigram and Trigram
Bigram Vocab:  221770
Trigram Vocab:  537462

E: min_df=10
min_df=10 vocab size: 3064

F: diff between train and dev data vocab
dev data vocab size: 16246
train data vocab size: 26879
train_data vocab size - dev_data vocab size:  10633



(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [105]:
def P3():
### STUDENT START ###
    kn_clf = Pipeline([('vect', CountVectorizer()),
                         ('knclf', KNeighborsClassifier())]);
    kn_clf = kn_clf.fit(train_data, train_labels)
    pred = kn_clf.predict(test_data)
    print metrics.f1_score(test_labels, pred, average=None)
    print metrics.accuracy_score(test_labels,pred)
#a
#b
    mnb_clf = Pipeline([('vect', CountVectorizer()),
                         ('mnclf', MultinomialNB())]);
    mnb_clf = mnb_clf.fit(train_data, train_labels)
    pred = mnb_clf.predict(test_data)
    print metrics.f1_score(test_labels, pred, average='weighted')
    print metrics.accuracy_score(test_labels,pred)

#c
    log_clf = Pipeline([('vect', CountVectorizer()),
                         ('lgclf', LogisticRegression(C=0.1))]);
    log_clf = log_clf.fit(train_data, train_labels)
    pred = log_clf.predict(test_data)
    print metrics.f1_score(test_labels, pred, average='weighted')
    print metrics.accuracy_score(test_labels,pred)
    weights = log_clf.named_steps['lgclf'].coef_


### STUDENT END ###
P3()

[ 0.37597911  0.48396501  0.43148688  0.23859649]
0.388478581979
0.767708659611
0.766617429838
0.737963877664
0.744460856721


ANSWER:

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [22]:
def P4():
### STUDENT START ###
    log_clf = Pipeline([('vect', CountVectorizer()),
                         ('lgclf', LogisticRegression())]);
    log_clf = log_clf.fit(train_data, train_labels)
    pred = log_clf.predict(test_data)
    print(metrics.classification_report(test_labels, pred,
               target_names=categories))
    print metrics.accuracy_score(test_labels,pred)
    
    weights = log_clf.named_steps['lgclf'].coef_
    sorted_weights = [];
    for w in range(weights.shape[0]):
            sorted_weights.append(sorted(weights[w],reverse=True))
    print 'label 1', '\t','label 2', '\t','label 3', '\t','label 4'
    for row in range(20):
        print sorted_weights[0][row], '\t',sorted_weights[1][row],'\t',sorted_weights[2][row],'\t',sorted_weights[3][row]
    
    print ''
    
    print 'bigram features: '
    bigram_vectorizer = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                         ('lgclf', LogisticRegression())]);
    bigram_vectorizer = bigram_vectorizer.fit(train_data, train_labels)
    pred = bigram_vectorizer.predict(test_data)
    print(metrics.classification_report(test_labels, pred,
               target_names=categories))
    print metrics.accuracy_score(test_labels,pred)
    
    weights = bigram_vectorizer.named_steps['lgclf'].coef_
    sorted_weights = [];
    for w in range(weights.shape[0]):
            sorted_weights.append(sorted(weights[w],reverse=True))
    print 'label 1', '\t','label 2', '\t','label 3', '\t','label 4'
    for row in range(20):
        print sorted_weights[0][row], '\t',sorted_weights[1][row],'\t',sorted_weights[2][row],'\t',sorted_weights[3][row]
    
    print ''

        
### STUDENT END ###
P4()

                    precision    recall  f1-score   support

       alt.atheism       0.64      0.60      0.62       154
talk.religion.misc       0.82      0.89      0.85       204
     comp.graphics       0.78      0.80      0.79       195
         sci.space       0.64      0.56      0.59       124

       avg / total       0.73      0.74      0.73       677

0.738552437223
label 1 	label 2 	label 3 	label 4
1.12508177938 	1.93657411939 	2.16258120443 	1.14737513368
1.0314304977 	1.34593184196 	1.22509675032 	1.11695669401
0.989697477191 	1.26621564383 	1.01142794935 	1.05460928274
0.953899227946 	1.12493558979 	0.936564906735 	0.912752341782
0.93998190407 	0.977813995928 	0.920343476009 	0.904666184262
0.869636439326 	0.910184128431 	0.825435807612 	0.825315541667
0.799656442115 	0.871665765913 	0.802353691448 	0.757832730994
0.789921015443 	0.851600252012 	0.751634319108 	0.73180960665
0.788987127133 	0.82728686158 	0.709321173164 	0.693178931693
0.779978352769 	0.821169562541 	0.70

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [39]:
def empty_preprocessor(s):
    return s

def better_preprocessor(s):
### STUDENT START ###
    repl = re.sub('&', 'and', s)
    repl = repl.lower()
    repl = repl.replace('0',' zero ')
    repl = repl.replace('1',' one ')
    repl = repl.replace('2',' two ')
    repl = repl.replace('3',' three ')
    repl = repl.replace('4',' four ')
    repl = repl.replace('5',' five ')
    repl = repl.replace('6',' six ')
    repl = repl.replace('7',' seven ')
    repl = repl.replace('8',' eight ')
    repl = repl.replace('9',' nine ')
    repl = re.sub('[^a-zA-Z]+', ' ', repl)
    return repl

### STUDENT END ###

def P5():
### STUDENT START ###
    log_clf = Pipeline([('vect', CountVectorizer(preprocessor=better_preprocessor)),
                         ('lgclf', LogisticRegression())]);
    log_clf = log_clf.fit(train_data, train_labels)
    pred = log_clf.predict(test_data)
    print(metrics.classification_report(test_labels, pred,
               target_names=categories))
    print metrics.accuracy_score(test_labels,pred)
    print 'vocab size:', len(log_clf.named_steps['vect'].vocabulary_)
### STUDENT END ###
P5()

                    precision    recall  f1-score   support

       alt.atheism       0.64      0.62      0.63       154
talk.religion.misc       0.87      0.84      0.85       204
     comp.graphics       0.74      0.85      0.79       195
         sci.space       0.65      0.57      0.61       124

       avg / total       0.74      0.74      0.74       677

0.741506646972
vocab size: 24029


(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [48]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###
    log_clf = Pipeline([('vect', CountVectorizer()),
                         ('lgclf', LogisticRegression(penalty='l1', C=0.1, tol=0.1))]);
    log_clf = log_clf.fit(train_data, train_labels)
    pred = log_clf.predict(test_data)
    print(metrics.classification_report(test_labels, pred,
               target_names=categories))
    print metrics.accuracy_score(test_labels,pred)
    weights = log_clf.named_steps['lgclf'].coef_
    
    ### STUDENT END ###
P6()

                    precision    recall  f1-score   support

       alt.atheism       0.60      0.56      0.58       154
talk.religion.misc       0.74      0.88      0.81       204
     comp.graphics       0.68      0.77      0.72       195
         sci.space       0.69      0.37      0.48       124

       avg / total       0.68      0.68      0.67       677

0.683899556869


(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [10]:
def P7():
    ### STUDENT START ###
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfid_vec = Pipeline([('tfidvec', TfidfVectorizer()),
                        ('log_clf', LogisticRegression(C=100))])
    tfid_vec = tfid_vec.fit(train_data, train_labels)
    pred = tfid_vec.predict(dev_data)
    print(metrics.classification_report(dev_labels, pred,
               target_names=categories))
    print metrics.accuracy_score(dev_labels,pred)

    ### STUDENT END ###
P7()

                    precision    recall  f1-score   support

       alt.atheism       0.69      0.62      0.65       165
talk.religion.misc       0.80      0.91      0.85       185
     comp.graphics       0.83      0.83      0.83       199
         sci.space       0.68      0.64      0.66       127

       avg / total       0.76      0.76      0.76       676

0.763313609467


ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance